In [144]:
import numpy as np
import pyphi #pyphi feature IIT.4 branch (quantum_python3.9 environment)

from qutip import *
from qutip_qip.operations import *
from intrinsic_difference import intrinsic_difference
from utils import *
from compute_ces import *

In [145]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Define states and matrices

In [146]:
# Basis states (for both classical and quantum)
zero = basis(2,0)
one = basis(2,1)

# Normalized fully general quantum state
plus = (zero + one).unit()
minus = (zero - one).unit()
superposition = (1/3*zero + 2/3*one).unit()

# Density matrix representations of the states
rho_zero = zero * zero.dag() # could also be fock_dm(2,0)
rho_one = one * one.dag()
rho_plus = plus * plus.dag()
rho_minus = minus * minus.dag()
rho_sp = superposition * superposition.dag()
rho_mm = 0.5 * rho_zero + 0.5 * rho_one

rho_bell = ket2dm(bell_state(state = '00'))
rho_GHZ = ket2dm(ghz_state(N=3))
rho_W = ket2dm(w_state(N=3))
rho_bellmix = 1/3*(tensor(rho_bell, rho_zero) + tensor(rho_zero, rho_bell) + tensor(rho_zero, rho_bell).permute([1,0,2]))

rho_bennet = 1/(3-4)*(1-(tensor(rho_zero, rho_one, rho_plus) + tensor(rho_one, rho_plus, rho_zero) 
                        + tensor(rho_plus, rho_zero, rho_one) + tensor(rho_minus, rho_minus, rho_minus)))


In [147]:
print('ID |0> : ', intrinsic_difference(rho_zero, rho_mm))
print('ID |+> : ', intrinsic_difference(rho_plus, rho_mm))

ID |0> :  (1.0, [array([1.+0.j, 0.+0.j])])
ID |+> :  (1.0, [array([0.70710678+0.j, 0.70710678+0.j])])


# 2 Qubits - CNOT

In [148]:
cnot()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

## Classical results

In [149]:
pyphi.config.PARTITION_TYPE = 'ALL'

In [150]:
print(pyphi.config)

{ 'ACTUAL_CAUSATION_MEASURE': 'PMI',
  'ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS': False,
  'CACHE_POTENTIAL_PURVIEWS': True,
  'CACHE_REPERTOIRES': True,
  'CES_DISTANCE': 'SUM_SMALL_PHI',
  'CLEAR_SUBSYSTEM_CACHES_AFTER_COMPUTING_SIA': False,
  'COMPOSITIONAL_STATE_CONFLICTS': 'GLOBAL',
  'CONGRUENCE_RATIO': 'PURVIEW_SIZE',
  'DISTINCTION_PHI_UPPER_BOUND_RELATIONS': 'PURVIEW_SIZE',
  'DISTINCTION_SUM_PHI_UPPER_BOUND': 'DISTINCT_AND_CONGRUENT_PURVIEWS',
  'HORIZONTAL_PARTITION_CODE': '1210',
  'IIT_VERSION': 'maximal-state-first',
  'LOG_FILE': 'pyphi.log',
  'LOG_FILE_LEVEL': 'INFO',
  'LOG_STDOUT_LEVEL': 'WARNING',
  'MAXIMUM_CACHE_MEMORY_PERCENTAGE': 50,
  'MICE_TIE_RESOLUTION': 'MAX_INFORMITAVENESS_THEN_SMALLEST_PURVIEW',
  'NUMBER_OF_CORES': -1,
  'OVERLAP_RATIO': 'PURVIEW_SIZE',
  'PARALLEL_COMPLEX_EVALUATION': True,
  'PARALLEL_COMPOSITIONAL_STATE_EVALUATION': True,
  'PARALLEL_CONCEPT_EVALUATION': True,
  'PARALLEL_CUT_EVALUATION': True,
  'PARTITION_TYPE': 'ALL',
  'PRECISION':

In [151]:
CNOT_tpm = cnot().data.toarray()
#CNOT_tpm = pyphi.convert.le2be_state_by_state(CNOT_tpm.real)

In [152]:
CNOT_tpm.real

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [153]:
# pyphi convention for state ordering is little endian
CNOT_tpm = pyphi.convert.be2le_state_by_state(CNOT_tpm.real)

In [154]:
network = pyphi.Network(CNOT_tpm)

In [157]:
# irreducible effects
subsystem_e = pyphi.Subsystem(network, (1,0))
mechs = pyphi.utils.powerset(range(len(network)), nonempty = True)
effects = [subsystem_e.find_mice(pyphi.Direction.EFFECT, m) for m in mechs if subsystem_e.find_mice(pyphi.Direction.EFFECT, m).phi>0]
effects

[Maximally-irreducible effect
   φ = 1
   Mechanism: [n0]
   Purview: [n0]
   Specified state(s): 
     [1]
   Direction: EFFECT
   MIP:
     n0     ∅ 
     ─── ✕ ───
      ∅    n0 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    0      │
     │ 1 *  1      │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1 *  1/2    │
     └─────────────┘,
 Maximally-irreducible effect
   φ = 1
   Mechanism: [n0,n1]
   Purview: [n1]
   Specified state(s): 
     [1]
   Direction: EFFECT
   MIP:
     n0,n1    ∅ 
     ───── ✕ ───
       ∅     n1 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    0      │
     │ 1 *  1      │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1 *  1/2    │
     └─────────────┘]

In [159]:
effects[1].ties

<bound method MaximallyIrreducibleCauseOrEffect.ties of Maximally-irreducible effect
  φ = 1
  Mechanism: [n0,n1]
  Purview: [n1]
  Specified state(s): 
    [1]
  Direction: EFFECT
  MIP:
    n0,n1    ∅ 
    ───── ✕ ───
      ∅     n1 
  Repertoire:
    ┌─────────────┐
    │ S    Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴ │
    │ 0    0      │
    │ 1 *  1      │
    └─────────────┘
  Partitioned repertoire:
    ┌─────────────┐
    │ S    Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴ │
    │ 0    1/2    │
    │ 1 *  1/2    │
    └─────────────┘>

In [161]:
# irreducible effects
subsystem_c = pyphi.Subsystem(network, (1,1))
mechs = pyphi.utils.powerset(range(len(network)), nonempty = True)
causes = [subsystem_c.find_mice(pyphi.Direction.CAUSE, m) for m in mechs if subsystem_c.find_mice(pyphi.Direction.CAUSE, m).phi>0]
causes

[Maximally-irreducible cause
   φ = 1
   Mechanism: [n0]
   Purview: [n0]
   Specified state(s): 
     [1]
   Direction: CAUSE
   MIP:
     n0     ∅ 
     ─── ✕ ───
      ∅    n0 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    0      │
     │ 1 *  1      │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1 *  1/2    │
     └─────────────┘,
 Maximally-irreducible cause
   φ = 1/2
   Mechanism: [n1]
   Purview: [n0,n1]
   Specified state(s): 
     [1, 0]
     [0, 1]
   Direction: CAUSE
   MIP:
     n1      ∅  
     ─── ✕ ─────
      ∅    n0,n1
   Repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    0      │
     │ 10 *  1/2    │
     │ 01 *  1/2    │
     │ 11    0      │
     └──────────────┘
   Partitioned repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    1/4    │
     │ 10 *  

In [164]:
causes[2].ties

<bound method MaximallyIrreducibleCauseOrEffect.ties of Maximally-irreducible cause
  φ = 1
  Mechanism: [n0,n1]
  Purview: [n0,n1]
  Specified state(s): 
    [1, 0]
  Direction: CAUSE
  MIP:
    n0     n1      ∅ 
    ─── ✕ ───── ✕ ───
     ∅    n0,n1    ∅ 
  Repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    0      │
    │ 10 *  1      │
    │ 01    0      │
    │ 11    0      │
    └──────────────┘
  Partitioned repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    0      │
    │ 10 *  1/2    │
    │ 01    1/2    │
    │ 11    0      │
    └──────────────┘>

In [167]:
subsystem_e.find_mice(pyphi.Direction.EFFECT, (0,1), ((0,1),))

Maximally-irreducible effect
  φ = 1
  Mechanism: [n0,n1]
  Purview: [n0,n1]
  Specified state(s): 
    [1, 1]
  Direction: EFFECT
  MIP:
     n0     n1     ∅ 
    ───── ✕ ─── ✕ ───
    n0,n1    ∅     ∅ 
  Repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    0      │
    │ 10    0      │
    │ 01    0      │
    │ 11 *  1      │
    └──────────────┘
  Partitioned repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    0      │
    │ 10    1/2    │
    │ 01    0      │
    │ 11 *  1/2    │
    └──────────────┘

## Quantum results

In [168]:
qgate = cnot()
#qgate = tensor(qeye(2),qeye(2),qeye(2)) 

In [186]:
#rho = tensor(rho_one, rho_zero)
#rho = tensor(rho_minus, rho_plus)
#rho = tensor(rho_plus, rho_zero)
rho = 0.5*tensor(rho_one, rho_one) + 0.5*tensor(rho_zero, rho_zero)

In [187]:
rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0.5]]

In [188]:
rho_e = evolve_unitary(rho, qgate, 'effect')
rho_e

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  0. ]]

In [193]:
result_e = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  1.0
p:  (0, 1)  phi:  0.5


In [194]:
result_e

[{'mech': (0, 1),
  'purview': (1,),
  'phi': 1.0,
  'mip': 0,1    ∅ 
  ─── ✕ ───
   ∅     1 ,
  'state': [array([1.+0.j, 0.+0.j])]}]

In [195]:
result_c = compute_ces(rho_e, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.5
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0


In [196]:
result_c

[{'mech': (1,),
  'purview': (0, 1),
  'phi': 0.5,
  'mip':  1     ∅ 
  ─── ✕ ───
   ∅    0,1,
  'state': [array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
   array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])]}]

In [185]:
Qobj(result_e[0]['state'][0])

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]]

In [248]:
bell_state(state = '10')

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]]

# 3 Qubits

In [228]:
#qgate = tensor(qeye(2), cnot())
#qgate = fredkin()
qgate = Qobj(qeye(8), dims = [[2, 2, 2], [2, 2, 2]])

# Quantum OR
# qgate = Qobj([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,1,0,0,0,0,0], 
#          [0,0,0,0,0,1,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,0,0,0,1,0]], dims = [[2, 2, 2], [2, 2, 2]])
# Quantum OR, c and b switched
# qgate = Qobj([[1,0,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,1,0,0,0,0,0,0], 
#          [0,0,0,0,0,0,1,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,0,0,1,0,0]], dims = [[2, 2, 2], [2, 2, 2]])


In [229]:
qgate

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]

In [233]:
#rho = rho_GHZ
#rho = tensor(rho_zero, rho_zero, rho_zero)
# rho = ket2dm((ket((1,1,0)) + ket((0,0,1))).unit())
rho = rho_W
rho

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]

In [234]:
rho_e = evolve_unitary(rho, qgate, 'effect')
rho_e

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]

In [235]:
result_e = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0.276692
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.138346
p:  (0, 2)  phi:  0.138346
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0.069173
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0.276692
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.138346
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0.138346
p:  (0, 1, 2)  phi:  0.069173
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0.276692
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0.138346
p:  (1, 2)  phi:  0.138346
p:  (0, 1, 2)  phi:  0.069173
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.943358
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0.471679
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0.943358
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0.471679
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0.943358
p:  (0, 1, 2)  phi:  0

In [242]:
result_e

[{'mech': (0,),
  'purview': (0,),
  'phi': 0.276692,
  'mip':  0     ∅ 
  ─── ✕ ───
   ∅     0 ,
  'state': [array([1.+0.j, 0.+0.j])]},
 {'mech': (1,),
  'purview': (1,),
  'phi': 0.276692,
  'mip':  1     ∅ 
  ─── ✕ ───
   ∅     1 ,
  'state': [array([1.+0.j, 0.+0.j])]},
 {'mech': (2,),
  'purview': (2,),
  'phi': 0.276692,
  'mip':  2     ∅ 
  ─── ✕ ───
   ∅     2 ,
  'state': [array([1.+0.j, 0.+0.j])]},
 {'mech': (0, 1),
  'purview': (0, 1),
  'phi': 0.943358,
  'mip': 0,1    ∅ 
  ─── ✕ ───
   ∅    0,1,
  'state': [array([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j, 0.        +0.j])]},
 {'mech': (0, 2),
  'purview': (0, 2),
  'phi': 0.943358,
  'mip': 0,2    ∅ 
  ─── ✕ ───
   ∅    0,2,
  'state': [array([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j, 0.        +0.j])]},
 {'mech': (1, 2),
  'purview': (1, 2),
  'phi': 0.943358,
  'mip': 1,2    ∅ 
  ─── ✕ ───
   ∅    1,2,
  'state': [array([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j, 0.        +0.j])]},
 {'mech': (0, 1, 2),
 

In [222]:
result_c = compute_ces(rho_e, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0.5
p:  (0, 1, 2)  phi:  0.25
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.5
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  1.0
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p

In [224]:
result_c

[{'mech': (2,),
  'purview': (1, 2),
  'phi': 0.5,
  'mip':  2     ∅ 
  ─── ✕ ───
   ∅    1,2,
  'state': [array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
   array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])]},
 {'mech': (0, 1),
  'purview': (0, 1, 2),
  'phi': 1.0,
  'mip': 0,1     ∅  
  ─── ✕ ─────
   ∅    0,1,2,
  'state': [array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j]),
   array([0.        +0.j, 0.70710678+0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j, 0.        +0.j, 0.70710678+0.j, 0.        +0.j])]},
 {'mech': (0, 1, 2),
  'purview': (0, 1, 2),
  'phi': 1.0,
  'mip':  0,1     2     ∅ 
  ───── ✕ ─── ✕ ───
  0,1,2    ∅     ∅ ,
  'state': [array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j])]}]

In [244]:
Qobj(result_e[3]['state'][0])

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]]

In [243]:
ptrace(rho_W, (0,))

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.66666667 0.        ]
 [0.         0.33333333]]

# Measurement

In [206]:
qgate = [tensor(rho_zero, qeye(2)), tensor(rho_one, qeye(2))]
#qgate = tensor(rho_zero, qeye(2))

In [111]:
qgate

[Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]],
 Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
 Qobj data =
 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]

In [119]:
rho = tensor(rho_sp, rho_mm)
#rho = rho_bell
rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.1 0.  0.2 0. ]
 [0.  0.1 0.  0.2]
 [0.2 0.  0.4 0. ]
 [0.  0.2 0.  0.4]]

In [120]:
kraus_to_super(qgate).iscptp

True

In [134]:
rho_m = evolve_cptp(rho, qgate, 'effect')
rho_m.ptrace(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.2 0. ]
 [0.  0.8]]

In [122]:
evolve_cptp(rho_m, qgate, 'cause')

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.1 0.  0.  0. ]
 [0.  0.1 0.  0. ]
 [0.  0.  0.4 0. ]
 [0.  0.  0.  0.4]]

In [135]:
result = compute_ces(tensor(rho_one, rho_mm), qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  1.0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.5
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0


In [136]:
result

[{'mech': (0,),
  'purview': (0,),
  'phi': 1.0,
  'mip':  0     ∅ 
  ─── ✕ ───
   ∅     0 ,
  'state': [array([0.+0.j, 1.+0.j])]}]

In [103]:
result_c = compute_ces(rho_m, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  0.542458
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.271229
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0


In [104]:
result_c

[{'mech': (0,),
  'purview': (0,),
  'phi': 0.542458,
  'mip':  0     ∅ 
  ─── ✕ ───
   ∅     0 ,
  'state': [array([0.+0.j, 1.+0.j])]}]

In [132]:
rho_sp

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.2 0.4]
 [0.4 0.8]]

In [133]:
measurement_statistics(rho, qgate)

([Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[0.5 0.  0.  0. ]
   [0.  0.5 0.  0. ]
   [0.  0.  0.  0. ]
   [0.  0.  0.  0. ]],
  Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[0.  0.  0.  0. ]
   [0.  0.  0.  0. ]
   [0.  0.  0.5 0. ]
   [0.  0.  0.  0.5]]],
 [0.19999999999999998, 0.7999999999999999])